In [49]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from datetime import date
import pandas as pd
import ssl
import math
import unidecode
import csv
import pandas
import requests
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from basketball_reference_web_scraper import client
console = "integratedTerminal"

# define the categories we need in the order we want for our datasets
cat_list = ['+/-', '3P', '3P%', '3PA', 'AST', 'BLK', 'DRB', 'FG', 'FG%', 'FGA', 'FT', 'FT%', 'FTA', 'GS', 'GmSc', 'ORB', 'PF', 'PTS', 'STL', 'TOV', 'TRB']

pct_cat_list = ['3P%',  'FG%', 'FT%']

## Define Function for pulling Player ID's and Player Name's
## ---------------------------------------------------

In [50]:
# Get data for all NBA players
def load_data(year):
    data = client.players_season_totals(season_end_year=year)
    nba = pd.DataFrame(data=data, index=None)
    return nba


In [51]:
# Remove duplicate NBA player names, and create player_id and player_name variables 
nba_data = load_data(2022)
player_repository = nba_data.drop_duplicates(subset = ['slug'])
player_id = player_repository.slug
player_name = player_repository.name
player_id_dictionary = dict(zip(player_repository.slug, player_repository.name))

## Define Function for scraping NBA Player Season Avg's

In [52]:
def cleaned_season_avg(pull_year):
    
    def scrape_player_season_avg(pull_year):
        
        year = pull_year
        url = "https://www.basketball-reference.com/leagues/NBA_{}_per_game.html".format(year)

        html = urlopen(url)

        soup = BeautifulSoup(html, "html.parser")

        soup.findAll('tr', limit=2)
        headers = [th.getText() for th in soup.findAll('tr', limit=2)[0].findAll('th')]

        headers = headers[1:]
        headers

        rows = soup.findAll('tr')[1:]
        player_stats = [[td.getText() for td in rows[i].findAll('td')]
                    for i in range(len(rows))]

        soup.findAll('tr', limit=2)
        headers = [th.getText() for th in soup.findAll('tr', limit=2)[0].findAll('th')]

        headers = headers[1:]
        headers

        rows = soup.findAll('tr')[1:]
        player_stats2 = []

        for i in range(len(rows)):
            row_text = []
            for td in rows[i].findAll('td'):
                row_text.append(td.getText())
            player_stats2.append(list(row_text))


        stats = pd.DataFrame(player_stats, columns = headers)
        stats = stats.rename(columns = {"FG%": "FGprc", "FT%": "FTprc"})
        stats["FGprc"] = "0" + stats["FGprc"]

        stats["FTprc"] = "0" + stats["FTprc"]
        stats = stats.loc[~(stats["Player"].isna())].copy() 
        
        return stats
    
    league_gamelog = pd.DataFrame()
    player_iter_count = 0

    stats = scrape_player_season_avg(pull_year)

    for player_selkt in player_name:
        test_season_avg_stats = stats.loc[stats['Player'] == player_selkt]
        
        if 'TOT' in test_season_avg_stats.Tm.values:
            test_season_avg_stats = test_season_avg_stats.loc[test_season_avg_stats['Tm'] == "TOT"]   
        else:
            test_season_avg_stats = test_season_avg_stats
        

        league_gamelog = pd.concat([league_gamelog, test_season_avg_stats])

    stats_2 = league_gamelog
    stats_2 = stats_2.rename(columns={'FGprc': 'FG%', 'FTprc': 'FT%', 'FGprc_Z': 'FG%_Z', 'FTprc_Z': 'FT%_Z', })
    
    print("cleaned_season_avg = success")
    return stats_2

## Define Function for pulling Gamelog Data from the Last 5 Games


In [53]:
def last_5_games_loop(pull_year):

    def last_5_games(playerid_pull, pull_year):
        
        pull_gamelog = requests.get('https://widgets.sports-reference.com/wg.fcgi?css=1&site=bbr&url=%2Fplayers%2Fh%2F{}%2Fgamelog%2F{}&div=div_pgl_basic'.format(playerid_pull, pull_year))
    
        if pull_gamelog.status_code==200:
            soup = BeautifulSoup(pull_gamelog.content, 'html.parser')
            table = soup.find('table')
        try:
            player_gamelog = pd.read_html(str(table))[0]
        except:
            return 
        
        #get player name for each playerid to append dataframe
        temp_player_id = playerid_pull
        playername = player_id_dictionary.get(temp_player_id)
        
        playerlist = []

    
        global player_iter_count
        player_iter_count += 1
        print(player_iter_count, playerid_pull)

        for row in player_gamelog.iterrows():
            playerlist += playername
    
        player_gamelog.insert(0,"Player ID", playerid_pull,True)
        player_gamelog.insert(1,"Player Name", playername,True)
        
        #remove DNP games so that we can call the last 5 games the player actually played in
        player_gamelog = player_gamelog[~player_gamelog['PTS'].isin(['Did Not Dress', 'Did Not Play', 'Inactive', 'Not With Team', 'PTS', 'Player Suspended'])]

        player_gamelog = player_gamelog.iloc[-5:]

        return player_gamelog

    global player_iter_count
    player_iter_count = 0
    last_5_gamelog = pd.DataFrame()
    
    
    for player_selkt in player_id:
        test_gametable = last_5_games(player_selkt, pull_year)

        last_5_gamelog = pd.concat([last_5_gamelog, test_gametable])
    
    last_5_gamelog = last_5_gamelog[['Player ID', 'Player Name', '+/-', '3P', '3P%', '3PA', 'AST', 'Age', 'BLK', 'DRB', 'Date', 'FG', 'FG%', 'FGA', 'FT', 'FT%', 'FTA', 'G', 'GS', 'GmSc', 'MP', 'ORB', 'Opp', 'PF', 'PTS', 'Rk', 'STL', 'TOV', 'TRB', 'Tm', 'Unnamed: 5', 'Unnamed: 7']]

    for i in cat_list:
        last_5_gamelog[i] = last_5_gamelog[i].map(float)
    print("last_5_games_loop = success")
    return last_5_gamelog

## Define function for calculating averages over last x number of games

In [54]:
def last_5_avg_loop(last_5_df):

    def last_5_avg(gamelog_df, cat_list, player_name):
        
        player_info = {'Player_Name': [player_name], 'PTS': [0]}  
        player_df = pd.DataFrame(player_info, columns = ['Player_Name', 'PTS'])
        Num_of_games = len(gamelog_df)


        for cat_selkt in cat_list:
            
            total_col = gamelog_df[cat_selkt].sum()
            avg_col = total_col/Num_of_games
            player_df[cat_selkt] = avg_col


        if player_df['3P'].sum() == 0: 
            player_df['3P%'] = 0
        else: 
            player_df['3P%'] = player_df['3P'].sum()/player_df['3PA'].sum()
        
        if player_df['FG'].sum() == 0:
            player_df['FG%'] == 0
        else:
            player_df['FG%'] = player_df['FG'].sum()/player_df['FGA'].sum()

        
        if player_df['FT'].sum() == 0:
            player_df['FT%'] == 0
        else:
            player_df['FT%'] = player_df['FT'].sum()/player_df['FTA'].sum()
        
        return player_df

    combined_player_avg_df = pd.DataFrame()

    for player_selkt in player_name:
        player_last_5 = last_5_df.loc[last_5_df['Player Name'] == player_selkt].copy()
        player_last_5_avg = last_5_avg(player_last_5, cat_list, player_selkt)    
        combined_player_avg_df = pd.concat([combined_player_avg_df, player_last_5_avg])
    print("last_5_avg_loop = success")
    return combined_player_avg_df

## Z-Score Calculator

In [55]:
#Z-Score Calculation 
def z_score_all(avg_stats_selkt):

    def z_score(temp_df, column_name):
        temp_dataframe = temp_df.copy()
        temp_dataframe[column_name] = temp_dataframe[column_name].map(float)
        column_mean = temp_dataframe[column_name].mean()
        
        temp_dataframe["Var_{0}".format(column_name)] = (temp_dataframe[column_name] - column_mean)**2
        column_std = math.sqrt(temp_dataframe["Var_{0}".format(column_name)].sum() / (temp_dataframe["Var_{0}".format(column_name)].count() -1))
        temp_dataframe["{0}_Z".format(column_name)] = (temp_dataframe[column_name] - column_mean)/ column_std
        temp_dataframe.drop(["Var_{0}".format(column_name)], axis = 1, inplace = True)
        return temp_dataframe

    column_name_list = ["FG%", "FT", "FT%", "3P", "PTS", "TRB", "AST", "STL", "BLK"]


    z_stats_temp = avg_stats_selkt
    for col_name_selkt in column_name_list:
        z_stats_temp = z_score(z_stats_temp, col_name_selkt)
    print("z_score_all = success")
    return z_stats_temp



## Define function for subtracting season avg. z-score from last 5 game avg. z-score to get net change

In [56]:
def z_score_diff_loop(z_season_temp, z_last_5_temp):

    def z_score_diff(z_season_temp, z_last_5_temp, player_name_selkt):
        temp_last_5_vs_season_avg = pd.DataFrame()
        player_nm_temp = player_name_selkt
        temp_season_avg = z_season_temp.loc[z_season_temp['Player'] == player_nm_temp].copy()
        temp_recent_games = z_last_5_temp.loc[z_last_5_temp['Player_Name'] == player_nm_temp].copy()
        
        column_name_list = ["FG%_Z", "FT_Z", "FT%_Z", "3P_Z", "PTS_Z", "TRB_Z", "AST_Z", "STL_Z", "BLK_Z"]
        
        for col_name_selkt in column_name_list:
            temp_last_5_vs_season_avg[col_name_selkt] = temp_recent_games[col_name_selkt] - temp_season_avg[col_name_selkt].reset_index(drop=True)

        
        temp_last_5_vs_season_avg.insert(0, "Player Name", player_nm_temp, True)

        temp_last_5_vs_season_avg["Total_Z"] = temp_last_5_vs_season_avg["FG%_Z"] + temp_last_5_vs_season_avg["FT_Z"] +temp_last_5_vs_season_avg["FT%_Z"] +temp_last_5_vs_season_avg["3P_Z"] + temp_last_5_vs_season_avg["PTS_Z"] + temp_last_5_vs_season_avg["TRB_Z"] + temp_last_5_vs_season_avg["AST_Z"] + temp_last_5_vs_season_avg["STL_Z"] + temp_last_5_vs_season_avg["BLK_Z"]

        return temp_last_5_vs_season_avg

    last_5_vs_season_z_score = pd.DataFrame()
    
    for player_name_selkt in player_name:
        pp = z_score_diff(z_season_temp, z_last_5_temp, player_name_selkt)
        last_5_vs_season_z_score = pd.concat([last_5_vs_season_z_score, pp])
    print("z_score_diff_loop = success")
    return last_5_vs_season_z_score

In [57]:
year_input = 2022

# Used in the Season Average Dashboard table
stats_2 = cleaned_season_avg(year_input)

# Used in the Gamelog (Last 5 Games) Dashboard table
last_5_gamelog = last_5_games_loop(year_input)

# Used in the Per Game Average (Last 5 Games) Dashboard table
last_5_avg = last_5_avg_loop(last_5_gamelog)

#Used as inputs for the Net Z-Score function below
z_score_last_5 = z_score_all(last_5_avg)
z_score_season_avg = z_score_all(stats_2)

# Used in the Net Z-Score (last 5 games vs season-to-date) Dashboard table
last_5_vs_season_z_score = z_score_diff_loop(z_score_season_avg, z_score_last_5)

cleaned_season_avg = success
1 achiupr01
2 adamsst01
3 adebaba01
4 aldamsa01
5 aldrila01
6 alexani01
7 allengr01
8 allenja01
9 alvarjo01
10 anderju01
11 anderky01
12 antetgi01
13 antetth01
14 anthoca01
15 anthoco01
16 anunoog01
17 arcidry01
18 arizatr01
19 augusdj01
20 avdijde01
21 ayayijo01
22 aytonde01
23 azubuud01
24 baglema01
25 ballla01
26 balllo01
27 bambamo01
28 banede01
29 bantoda01
30 barbeca01
31 barneha02
32 barnesc01
33 barrerj01
34 bartowi01
35 basspa01
36 bassech01
37 bateske01
38 batumni01
39 bazemke01
40 bazleda01
41 bealbr01
42 beaslma01
43 belljo01
44 bembrde01
45 bertada01
46 beverpa01
47 beysa01
48 birchkh01
49 bitadgo01
50 biyombi01
51 bjeline01
52 bledser01
53 blevike01
54 bogdabo01
55 bogdabo02
56 bolbo01
57 bolmale01
58 bongais01
59 bookede01
60 bostobr01
61 bouchch01
62 bouknja01
63 bradlav01
64 bradlto01
65 brazdig01
66 bridgmi01
67 bridgmi02
68 brissos01
69 brogdma01
70 brookar01
71 brookdi01
72 brownbr01
73 brownch02
74 brownch05
75 browngr01
76 brownja02
77

In [58]:
## Append Total_Z to season avg and last 5 avg. so that dashboard can sort to top Net-Z player
total_z_list = last_5_vs_season_z_score[['Player Name', 'Total_Z']]
stats_2 = stats_2.join(total_z_list.set_index('Player Name'), on = "Player")

last_5_avg = last_5_avg.join(total_z_list.set_index('Player Name'), on = "Player_Name")


## Export Data to Google Sheets via Google API

In [59]:
# To access spreadsheets (read, view, edit) via Google's API you need to set up a service account and add the file bath to your JSON key  
gc = gspread.service_account(filename=r'C:\Users\test_user\json_key_filename.json')

## Create empty spreadsheets called "last 5 games"

#Export Last 5 Games Avg vs Season Avg - Net Z-Score 
sh_last5 = gc.open("last 5 games")
worksheet_last5_vs_season_z = sh_last5.get_worksheet(0)
worksheet_last5_vs_season_z.resize(rows=1)
set_with_dataframe(worksheet_last5_vs_season_z, last_5_vs_season_z_score)

#Export Last 5 Games
sh_last5 = gc.open("last 5 games")
worksheet_last5_gamelog = sh_last5.get_worksheet(1)
worksheet_last5_gamelog.resize(rows=1)
set_with_dataframe(worksheet_last5_gamelog, last_5_gamelog)

#Export Last 5 Games Avg.
sh_last5 = gc.open("last 5 games")
worksheet_last5_avg = sh_last5.get_worksheet(2)
worksheet_last5_avg.resize(rows=1)
set_with_dataframe(worksheet_last5_avg, last_5_avg)

#Export Season-to-date Avg.
sh_last5 = gc.open("last 5 games")
worksheet_season_avg = sh_last5.get_worksheet(3)
worksheet_season_avg.resize(rows=1)
set_with_dataframe(worksheet_season_avg , stats_2)





Export Z-Score Analysis to CSV for Local Record Store

In [60]:
#Dynamically names CSV file based on runtime date
date.today()
today = date.today()
last_5_vs_season_z_score.to_csv(r".\..\extracts\last_5_z_vs_season_{}_all.csv".format(today))
